In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from google.colab import drive

#pd.set_option('display.max_columns', 500)
drive.mount('/content/gdrive')

mlb = MultiLabelBinarizer()

df = pd.read_json('gdrive/My Drive/csvs/cooking/train.json')
df_test = pd.read_json('gdrive/My Drive/csvs/cooking/test.json')
df_all = pd.concat([df_test, df])
h = df_test.shape[0]
t = df.shape[0]

df_dummies = pd.DataFrame(mlb.fit_transform(df_all['ingredients']), columns=mlb.classes_, index=df_all.index)
df_all = pd.concat([df_all['id'], df_all['cuisine'], df_dummies], axis=1)

df_test = df_all.head(h)
df = df_all.tail(t)
df_y = pd.get_dummies(df['cuisine'], columns=['cuisine'])

df = df.drop(['id', 'cuisine'], axis=1)
ids = df_test['id']
df_test = df_test.drop(['id', 'cuisine'], axis=1)

X, Y = df.values, df_y.values
X, Y = shuffle(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
X_kaggle = df_test.values

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [0]:
from ann_theano_model import ANN

model = ANN([200, 200])
model.fit(
    X_train, Y_train, X_test, Y_test,
    learning_rate = 1e-3,
    reg = 1e-4,
    epochs= 40,
    batch_size = 3331,
    print_period = 1,
    decay=0.995
)
model.score(X_test, Y_test)

Epoch  0 / 40 : Cost:  8764.510381378863  Classification rate:  0.2005942404388237
Epoch  1 / 40 : Cost:  9253.338016256832  Classification rate:  0.16021636446746915
Epoch  2 / 40 : Cost:  8909.824274829243  Classification rate:  0.19853725430443395
Epoch  3 / 40 : Cost:  8654.513848590774  Classification rate:  0.2715221697394484
Epoch  4 / 40 : Cost:  6937.913444129421  Classification rate:  0.4356239524607649
Epoch  5 / 40 : Cost:  8297.990393988632  Classification rate:  0.2288587536187719
Epoch  6 / 40 : Cost:  5599.10816516448  Classification rate:  0.504647264970288
Epoch  7 / 40 : Cost:  5010.92599141948  Classification rate:  0.5374828584488801
Epoch  8 / 40 : Cost:  4849.370258232507  Classification rate:  0.5577479811062014
Epoch  9 / 40 : Cost:  4313.69504925534  Classification rate:  0.5726039920767941
Epoch  10 / 40 : Cost:  4311.384996433059  Classification rate:  0.5752704555843364
Epoch  11 / 40 : Cost:  3723.4540196606677  Classification rate:  0.6006399512418101
Epo

KeyboardInterrupt: ignored

In [0]:
preds = model.predict(X_kaggle)
cuisines = df_y.columns.values
pred_to_cuisine = np.vectorize(lambda x: cuisines[x])
preds = pred_to_cuisine(preds)

data = np.concatenate((np.array([ids]).T, np.array([preds]).T), axis=1)
result = pd.DataFrame(data, columns = ['id', 'cuisine'])
result.to_csv('gdrive/My Drive/cooking_result.csv', index=False)